In [ ]:
import time
import math
import torch
import numpy as np
import quairkit as qkit
import matplotlib.pyplot as plt
import scipy

from typing import List, Tuple
from tqdm import tqdm
from quairkit.database.state import zero_state, ghz_state
from quairkit.database.matrix import *
from quairkit.database import *
from quairkit.core.state import to_state
from quairkit.circuit import Circuit
from quairkit.qinfo import permute_systems, partial_transpose, trace_distance, dagger, NKron

qkit.set_dtype('complex128')
NUMPY_DTYPE = 'complex128'

In [12]:
import scipy.linalg


def uhermitian_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    data_H = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in tqdm(range(num_samples)):
        # Randomly choose eigenvalues of +1 or -1
        eigenvalues = np.random.choice([1.0, -1.0], size=dim)
        D = np.diag(eigenvalues)
        # Generate a random unitary matrix U
        U = haar_unitary(dim)
        data_H[y, :, :] = U @ D @ U.conj().T
    return torch.tensor(data_H)


def random_diag_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    data_H = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in tqdm(range(num_samples)):
        # Randomly choose eigenvalues of +1 or -1
        eigenphase = 1*np.pi*np.random.random(size=dim)
        data_H[y, :, :] = np.diag(eigenphase)
    return torch.tensor(data_H)


def random_pauli_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    n = int(np.log2(dim))
    data_H = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    P = pauli_group(n)
    for y in tqdm(range(num_samples)):
        data_H[y, :, :] = P[np.random.randint(1, 4**n)]
    return torch.tensor(data_H)


def orthogonal_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    data_U = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in tqdm(range(num_samples)):
        data_U[y, :, :] = 1j*scipy.linalg.logm(haar_orthogonal(dim).numpy())
    return torch.tensor(data_U)


In [13]:
def compute_eigenspace_projectors(H:np.ndarray, tol=1e-6):
    """
    Compute the projection operators onto the eigenspaces of a Hermitian matrix H.
    
    Parameters:
    - H (ndarray): A Hermitian matrix.
    - tol (float): Tolerance for grouping nearly equal eigenvalues.
    
    Returns:
    - projectors (dict): A dictionary mapping each distinct eigenvalue
                         to its corresponding projector (ndarray).
    """
    
    # Diagonalize H; np.linalg.eigh is appropriate for Hermitian matrices.
    eigenvalues, eigenvectors = np.linalg.eigh(H)
    
    # Group indices of eigenvectors corresponding to (nearly) the same eigenvalue.
    projectors = {}
    # We'll keep track of "unique" eigenvalues up to the tolerance tol.
    unique_eigvals = []
    eig_indices = {}
    
    for idx, eig in enumerate(eigenvalues):
        matched = False
        for unique in unique_eigvals:
            if abs(eig - unique) < tol:
                matched = True
                eig_indices[unique].append(idx)
                break
        if not matched:
            unique_eigvals.append(eig)
            eig_indices[eig] = [idx]
    
    # Compute the projector for each unique eigenvalue.
    for eig in unique_eigvals:
        indices = eig_indices[eig]
        # Extract the eigenvectors (each column is an eigenvector)
        vecs = eigenvectors[:, indices]
        # Construct the projector onto the subspace: P = sum_i |v_i><v_i|
        projector = vecs @ vecs.conj().T
        projectors[eig] = projector
    
    return projectors


def joint_eigen_overlap(data:np.ndarray, rho_in:np.ndarray):
    '''Compute the joint eigenspace overlap of each individual tensor be given
    '''
    res = []
    for j in tqdm(range(data.shape[0])):
        val = 0
        for proj in compute_eigenspace_projectors(data[j]).values():
            val += np.trace(proj @ rho_in).real ** 2
        res.append(val)
    return res

In [ ]:
from scipy.linalg import logm

n_data = 1000
n_q = 7
# U_data = random_unitary(n_q, n_data).numpy()
# H_data = random_pauli_data_generation(2**n_q, n_data)
# H_data = uhermitian_data_generation(2**n_q, n_data)
# H_data = random_diag_data_generation(2**n_q, n_data)
# H_data = orthogonal_data_generation(2**n_q, n_data)

100%|██████████| 1000/1000 [01:09<00:00, 14.49it/s]


In [26]:
# cir = Circuit(n_q)
# cir.ry(param=[np.pi/4 for _ in range(n_q)])
# rho_in = cir().density_matrix.numpy()

rho_in = 0.9*zero_state(n_q).density_matrix.numpy() + 0.1*np.eye(2**n_q)/(2**n_q)

jeo = joint_eigen_overlap(H_data, rho_in)

100%|██████████| 1000/1000 [00:40<00:00, 24.45it/s]


In [27]:
np.mean(jeo)

0.01394985646096117

In [24]:
np.std(jeo)

0.0004533960291509382